In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
#changes made while training in order to replace the existing feature extraction layer of resnet model being used in the encodes with a vit(visual transformer ) based model
#the vit model expects square images which are of size 224*224 or preferabbly dimensions which are divisible by 16 else it wont work properly 
#however the existing code base allows us to modify the imgW (image width) and imgH(image Height) which are training parameters
#passed using parser and saved into the opt used
#further vit model doesnt support padding of images but by default train.py adds the padding due to line 
# parser.add_argument('--PAD', action='store_true', help='whether to keep ratio then pad for image resize') so insterad while training
#we need to pass custom value of -PAD action='store_false'instead during training as a training parameter
#thankfully no other files need to be modified and image preprocessing of tps which is put in place in the codebase can be followed as it is with these resizing parameters
#now the major modification is in featureextraction.py where the currently supported models for feature extraction have their codes written 
#the various models supported for feature extraction in the current codebase as of now are 
#VGG,RCNN,ResNet and GRCL 
#now we will add a custom class for vit where we will be training a vit_b_16 model from scratch which is not pretrained with these specification:
#Input size: 224×224 images
# Patch size: 16×16
# Embedding dim: 768
# Number of heads: 12
# Transformer depth: 12
# MLP size: 3072
# Output: 1000-dim logits
# ViT	Vision Transformer – a model architecture that applies Transformer layers (originally used in NLP) to image data.
# B	Base model size (ViT has different sizes: Tiny, Small, Base, Large).
# 16	Patch size – the image is divided into 16x16 patches before being fed into the transformer.
#the code added to featureextraction.py is as follows:
#from torchvision.models.vision_transformer import vit_b_16
#is added to the top of the featureextraction.py file of the working directory cloned repo file of featureextraction.py
#further a class of vitfeatureextraction code is also added to allow usage of vit model similar to resnet and other above listed models:
# class ViT_FeatureExtractor(nn.Module):
#     """ Vision Transformer Feature Extractor """

#     def __init__(self, input_channel=3, output_channel=512, image_size=224, patch_size=16):
#         super(ViT_FeatureExtractor, self).__init__()

#         # Initialize a base ViT
#         self.vit = vit_b_16(pretrained=False)
        
#         # Modify input channels if necessary (default ViT uses 3)
#         if input_channel != 3:
#             self.vit.conv_proj = nn.Conv2d(input_channel, self.vit.conv_proj.out_channels, kernel_size=patch_size, stride=patch_size)

#         # Optional: reduce to fixed output channels to match rest of pipeline
#         self.projector = nn.Linear(self.vit.heads.in_features, output_channel)
#         self.vit.heads = nn.Identity()  # Remove classification head

#     def forward(self, x):
#         x = self.vit(x)  # [batch, vit_dim]
#         x = self.projector(x)
#         x = x.unsqueeze(2).unsqueeze(3)  # [batch, channel, 1, 1] to match rest of pipeline
#         return x

#now once code is updated finally in train.py we need to provide opt with an option to use the newly created vit class consisting of vit moel for feature extraction similar to other models for which 
#we update the code of model.py code accordingly as below with this extra line:
# elif opt.FeatureExtraction == 'ViT':
#             self.FeatureExtraction = ViT_FeatureExtractor(opt.input_channel, opt.output_channel)
#and as usual to allow import of vit model add 
#from torchvision.models.vision_transformer import vit_b_16
#to top of the file
#further
#change this in model.py
# visual_feature = self.AdaptiveAvgPool(visual_feature.permute(0, 3, 1, 2))  # [b, c, h, w] -> [b, w, c, h]
# visual_feature = visual_feature.squeeze(3)
# to 
# if self.stages['Feat'] == 'ViT':
#     pass  # nothing to reshape # Already in [B, seq_len, C] format (like [B, 197, 768])
# else:
#     visual_feature = self.AdaptiveAvgPool(visual_feature.permute(0, 3, 1, 2))
#     visual_feature = visual_feature.squeeze(3)  # [B, C, T]
#to skip adaptiveavgpooling for vit model since it gives 1d token features
#further changes needed are as follows:
#in trin.py we will need to again pass custom paramteres for the default input and output_channel as for rgb images vit needs 3 input channels and 78 output_channels which is contrast to default paramteres
#default:
 # 
# change to 
# opt.input_channel = 3  # if using RGB images for ViT
# opt.output_channel = 768 
#just omit PAD entirely from training parameters to ensure pdding is not done
#pass --rgb as another training parameter to train on coloured images as well for better accuracy just pass it dont need to give it value as passing it sets it to true as per parser code in train.py 
#one more change 
#change this 
# self.FeatureExtraction = ViT_FeatureExtractor(input)
#to
# self.FeatureExtraction = ViT_FeatureExtractor(
#     input_channel=3,
#     output_channel=512,    # if you're projecting down to 512
#     image_size=224,
#     patch_size=16
# )
#modify train.py also and add the newly created model there as import statement at the top as follows:
#from model.modules.feature_extraction import ViT_FeatureExtractor
#to pass vit as feature extractor when training we need to pass parameter as 
#--FeatureExtraction ViT
#we also need to import the class defined newly of ViT_featureextractor onto our model.py so for that in model;py
#change this
#from modules.feature_extraction import VGG_FeatureExtractor, RCNN_FeatureExtractor, ResNet_FeatureExtractor
#to
#from modules.feature_extraction import ViT_FeatureExtractor,VGG_FeatureExtractor, RCNN_FeatureExtractor, ResNet_FeatureExtractor
#also in the vit feature extractor class made in feature_Extraction.py we need to make one change to follow hugging face rules:
#change this
#self.projector = nn.Linear(self.vit.heads.in_features, output_channel)
#to
#self.projector = nn.Linear(self.vit.hidden_dim, output_channel)
#forward function in model.py is 
# def forward(self, input, text, is_train=True):
        # """ Transformation stage """
        # if not self.stages['Trans'] == "None":
        #     input = self.Transformation(input)

        # if self.stages['Feat'] == 'ViT':
        #     visual_feature = self.FeatureExtraction(input)
           
        # else:
        #     visual_feature = self.AdaptiveAvgPool(visual_feature.permute(0, 3, 1, 2))  # [b, c, h, w] -> [b, w, c, h]
        #     visual_feature = visual_feature.squeeze(3)

        # """ Sequence modeling stage """
        # if self.stages['Seq'] == 'BiLSTM':
        #     contextual_feature = self.SequenceModeling(visual_feature)
        # else:
        #     contextual_feature = visual_feature  # for convenience. this is NOT contextually modeled by BiLSTM

        # """ Prediction stage """
        # if self.stages['Pred'] == 'CTC':
        #     prediction = self.Prediction(contextual_feature.contiguous())
        # else:
        #     prediction = self.Prediction(contextual_feature.contiguous(), text, is_train, batch_max_length=self.opt.batch_max_length)

        # return prediction
#due to memory considerations and visul transformers requireing more memory instead of using default batch size of 192 lets try 32 instead and pass it as a training parameter
#further we will reduce image size from 224x224 to 128x128 both in training parameters as well asd class definition in feature_extraction.py inside the __init__ parameters as well as 
#the forward function
#change 
#self.vit = vit_b_16(pretrained=False)
#to
#self.vit = vit_b_16(pretrained=False, image_size=128)
#final vit class
#class ViT_FeatureExtractor(nn.Module):
    # """ Vision Transformer Feature Extractor """

    # def __init__(self, input_channel=3, output_channel=512, image_size=128, patch_size=16):
    #     super(ViT_FeatureExtractor, self).__init__()

    #     # Initialize a base ViT
    #     self.vit = vit_b_16(pretrained=False, image_size=image_size)
        
    #     # Modify input channels if necessary (default ViT uses 3)
    #     if input_channel != 3:
    #         self.vit.conv_proj = nn.Conv2d(input_channel, self.vit.conv_proj.out_channels, kernel_size=patch_size, stride=patch_size)

    #     # Optional: reduce to fixed output channels to match rest of pipeline
    #     self.projector = nn.Linear(self.vit.hidden_dim, output_channel)
    #     self.vit.heads = nn.Identity()  # Remove classification head

    # def forward(self, x):
    #     x = self.vit(x)  # [batch, vit_dim, seq_len] (after transformer)
    #     x = x.flatten(1)  # Flatten the seq_len dimension, output shape: [batch, vit_dim, seq_len * feature_size]
    #     x = self.projector(x)  # Project to desired output channel size
    #     x=x.unsqueeze(2)
    #     x = x.permute(0, 2, 1)  # [batch, seq_len, vit_dim] (LSTM expects this)
    #     return x
#the first attempt at succesful training went horribly where earlystopping mechanism trigerred from the very second epoch iteslf so some major changes
#are needed to prevent the overfitting problem which was happening
#training loss was reducing but valid loss kept increasing 
#adding --adam parameter in training  to vary learning rate and keep reducing it to learn better
#self.dropout = nn.Dropout(0.2)
#adding dropout inside the vit clss made in feature_axtraction.py at end of init function
#current updates adding adam did not help and it doubled the number of epochs which had to be run
#major change making the vit class run on only 1 input channel instead of 3 as the IAM words dataset is greyscale image
#dataset and is not a coloured rgb dataset thus ,modifications introdcued is removing input_channel ,output_channel and rgb from training parameters as their default values of 1,512 and false are enough 
#changed the vit class code alone to get final code as 
# class ViT_FeatureExtractor(nn.Module):
#     """ Vision Transformer Feature Extractor (for Grayscale Images) """

#     def __init__(self, input_channel=1, output_channel=512, image_size=128, patch_size=16):
#         super(ViT_FeatureExtractor, self).__init__()

#         # Initialize base ViT
#         self.vit = vit_b_16(pretrained=False, image_size=image_size)

#         # Replace input conv layer for grayscale input
#         if input_channel != 3:
#             self.vit.conv_proj = nn.Conv2d(
#                 input_channel, 
#                 self.vit.conv_proj.out_channels, 
#                 kernel_size=patch_size, 
#                 stride=patch_size
#             )

#         # Dropout to prevent overfitting
#         self.dropout = nn.Dropout(0.2)

#         # Replace classifier head with projection to output_channel
#         self.vit.heads = nn.Identity()
#         self.projector = nn.Linear(self.vit.hidden_dim, output_channel)

#     def forward(self, x):
#         # x shape: [B, 1, H, W] for grayscale
#         x = self.vit(x)  # Output: [B, hidden_dim]
#         x = self.dropout(x)  # Apply dropout
#         x = self.projector(x)  # Project to desired output size
#         x = x.unsqueeze(2)  # [B, output_channel, 1]
#         x = x.permute(0, 2, 1)  # [B, seq_len=1, output_channel]
#         return x
#removed adam as a parameter also 
#dropout still remains it is applied in __init__ function as well as in forward function
#learning rate has been changed from default value of 1 which leads to overfitting into 0.1 for better and more smoother validation loss reduction results
#this training did not give good result for accuracy so one more change was made for training where along with lr of 0.1 also put a rho of 0.85 as well 
#that changed accuracy from 27 to 23 so even worse 
#now modified the class vit code again and instead of collapsing output of vit to one vecor allowed multiple vectors now 
# class ViT_FeatureExtractor(nn.Module):
#     """Vision Transformer Feature Extractor for Grayscale Images"""

#     def __init__(self, input_channel=1, output_channel=512, image_size=128, patch_size=16):
#         super(ViT_FeatureExtractor, self).__init__()

#         # Load ViT backbone without pretraining
#         self.vit = vit_b_16(pretrained=False, image_size=image_size)

#         # Modify patch embedding for grayscale
#         self.vit.conv_proj = nn.Conv2d(
#             input_channel, 
#             self.vit.conv_proj.out_channels, 
#             kernel_size=patch_size, 
#             stride=patch_size
#         )

    #     self.vit.heads = nn.Identity()  # Remove classification head

    #     self.dropout = nn.Dropout(0.2)

    #     self.projector = nn.Linear(self.vit.hidden_dim, output_channel)

    # def forward(self, x):
    # # x is [B, 1, H, W]
    # n, _, h, w = x.shape  # <--- this must be done before conv_proj changes the shape

    # x = self.vit.conv_proj(x)  # [B, hidden_dim, H', W']
    # x = x.flatten(2).transpose(1, 2)  # [B, N, hidden_dim]

    # x = x + self.vit.encoder.pos_embedding[:, 1:, :]  # Add positional encoding
    # x = self.vit.encoder.dropout(x)

    # x = self.vit.encoder.ln_pre(x)
    # for blk in self.vit.encoder.layers:
    #     x = blk(x)
    # x = self.vit.encoder.ln_post(x)

    # x = self.dropout(x)
    # x = self.projector(x)  # [B, N, output_channel]

    # return x
#also for next training removed rho =0.85 parameter and sticking to default of rho 0.95 itself 
#above code gave errors so trying below code:
# class ViT_FeatureExtractor(nn.Module):
#     """ Vision Transformer Feature Extractor (for Grayscale Images) """

#     def __init__(self, input_channel=1, output_channel=512, image_size=128, patch_size=16):
#         super(ViT_FeatureExtractor, self).__init__()

#         # Initialize base ViT (without pretrained weights)
#         self.vit = vit_b_16(pretrained=False, image_size=image_size)

#         # Replace input conv layer for grayscale input
#         if input_channel != 3:
#             self.vit.conv_proj = nn.Conv2d(
#                 input_channel, 
#                 self.vit.conv_proj.out_channels, 
#                 kernel_size=patch_size, 
#                 stride=patch_size
#             )

#         # Dropout to prevent overfitting
#         self.dropout = nn.Dropout(0.2)

#         # Replace the classifier head with an Identity layer
#         self.vit.heads = nn.Identity()

        # Project the output from the ViT encoder to the desired output channel
    #     self.projector = nn.Linear(self.vit.hidden_dim, output_channel)

    # def forward(self, x):
    #     # x shape: [B, 1, H, W] for grayscale (batch size, channels, height, width)
        
    #     # Pass through ViT backbone (transformer encoder)
    #     x = self.vit(x)  # Output shape: [B, seq_len, hidden_dim]

    #     # Apply dropout after the transformer encoder
    #     x = self.dropout(x)  # Shape: [B, seq_len, hidden_dim]

    #     # Project each token's feature vector to the desired output channel size
    #     x = self.projector(x)  # Shape: [B, seq_len, output_channel]
        
    #     # Now x contains a sequence of feature vectors (not collapsed to a single vector)
    #     return x  # Output shape: [B, seq_len, output_channel]
#for vit feature extraction further implemented label smoothing as a class and a scheduler also for learning rate which will half the learing rate if early stop reach count of 5
#further the output channels is 768 not 512 for vit model and lr is kept at 0.1 itself with adadelta
#now implementing a neuralodebased resnet feature extractor whose aim will be to reduce total number of training parameters needed as well improve backpropogation time


In [ ]:
!git clone https://github.com/dmitrijsk/AttentionHTR.git
!cd AttentionHTR

In [ ]:
!cp AttentionHTR/requirements.txt my_requirements.txt
!sed -i '/torch==1.9.0/d' my_requirements.txt  # Remove old PyTorch version
!pip install -r my_requirements.txt

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install lmdb

In [ ]:
!pip install torchdiffeq

In [ ]:

%%writefile /kaggle/working/AttentionHTR/model/dataset.py
import os
import sys
import re
import six
import math
import lmdb
import torch

from natsort import natsorted
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, ConcatDataset, Subset
from itertools import accumulate as _accumulate
import torchvision.transforms as transforms


class Batch_Balanced_Dataset(object):

    def __init__(self, opt):
        """
        Modulate the data ratio in the batch.
        For example, when select_data is "MJ-ST" and batch_ratio is "0.5-0.5",
        the 50% of the batch is filled with MJ and the other 50% of the batch is filled with ST.
        """
        log = open(f'./saved_models/{opt.exp_name}/log_dataset.txt', 'a')
        dashed_line = '-' * 80
        print(dashed_line)
        log.write(dashed_line + '\n')
        print(f'dataset_root: {opt.train_data}\nopt.select_data: {opt.select_data}\nopt.batch_ratio: {opt.batch_ratio}')
        log.write(f'dataset_root: {opt.train_data}\nopt.select_data: {opt.select_data}\nopt.batch_ratio: {opt.batch_ratio}\n')
        assert len(opt.select_data) == len(opt.batch_ratio)

        _AlignCollate = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
        self.data_loader_list = []
        self.dataloader_iter_list = []
        batch_size_list = []
        Total_batch_size = 0
        for selected_d, batch_ratio_d in zip(opt.select_data, opt.batch_ratio):
            _batch_size = max(round(opt.batch_size * float(batch_ratio_d)), 1)
            print(dashed_line)
            log.write(dashed_line + '\n')
            _dataset, _dataset_log = hierarchical_dataset(root=opt.train_data, opt=opt, select_data=[selected_d])
            total_number_dataset = len(_dataset)
            log.write(_dataset_log)

            """
            The total number of data can be modified with opt.total_data_usage_ratio.
            ex) opt.total_data_usage_ratio = 1 indicates 100% usage, and 0.2 indicates 20% usage.
            See 4.2 section in our paper.
            """
            number_dataset = int(total_number_dataset * float(opt.total_data_usage_ratio))
            dataset_split = [number_dataset, total_number_dataset - number_dataset]
            indices = range(total_number_dataset)
            _dataset, _ = [Subset(_dataset, indices[offset - length:offset])
                           for offset, length in zip(_accumulate(dataset_split), dataset_split)]
            selected_d_log = f'num total samples of {selected_d}: {total_number_dataset} x {opt.total_data_usage_ratio} (total_data_usage_ratio) = {len(_dataset)}\n'
            selected_d_log += f'num samples of {selected_d} per batch: {opt.batch_size} x {float(batch_ratio_d)} (batch_ratio) = {_batch_size}'
            print(selected_d_log)
            log.write(selected_d_log + '\n')
            batch_size_list.append(str(_batch_size))
            Total_batch_size += _batch_size

            _data_loader = torch.utils.data.DataLoader(
                _dataset, batch_size=_batch_size,
                shuffle=True,
                num_workers=int(opt.workers),
                collate_fn=_AlignCollate, pin_memory=True)
            self.data_loader_list.append(_data_loader)
            self.dataloader_iter_list.append(iter(_data_loader))

        Total_batch_size_log = f'{dashed_line}\n'
        batch_size_sum = '+'.join(batch_size_list)
        Total_batch_size_log += f'Total_batch_size: {batch_size_sum} = {Total_batch_size}\n'
        Total_batch_size_log += f'{dashed_line}'
        opt.batch_size = Total_batch_size

        print(Total_batch_size_log)
        log.write(Total_batch_size_log + '\n')
        log.close()

    def get_batch(self):
        balanced_batch_images = []
        balanced_batch_texts = []

        for i, data_loader_iter in enumerate(self.dataloader_iter_list):
            try:
                image, text = next(data_loader_iter)
                balanced_batch_images.append(image)
                balanced_batch_texts += text
            except StopIteration:
                self.dataloader_iter_list[i] = iter(self.data_loader_list[i])
                image, text = next(self.dataloader_iter_list[i])
                balanced_batch_images.append(image)
                balanced_batch_texts += text
            except ValueError:
                pass

        balanced_batch_images = torch.cat(balanced_batch_images, 0)

        return balanced_batch_images, balanced_batch_texts


def hierarchical_dataset(root, opt, select_data='/'):
    """ select_data='/' contains all sub-directory of root directory """
    dataset_list = []
    dataset_log = f'dataset_root:    {root}\t dataset: {select_data[0]}'
    print(dataset_log)
    dataset_log += '\n'
    for dirpath, dirnames, filenames in os.walk(root+'/'):
        if not dirnames:
            select_flag = False
            for selected_d in select_data:
                if selected_d in dirpath:
                    select_flag = True
                    break

            if select_flag:
                dataset = LmdbDataset(dirpath, opt)
                sub_dataset_log = f'sub-directory:\t/{os.path.relpath(dirpath, root)}\t num samples: {len(dataset)}'
                print(sub_dataset_log)
                dataset_log += f'{sub_dataset_log}\n'
                dataset_list.append(dataset)

                # Export predictions only when testing
                # because dir `/result/{opt.exp_name}` is created only during testing,
                # but this function is also be called by train.py.
                if hasattr(opt, 'eval_data'):
                    eval_dir = opt.eval_data.split("/")[-1]
                    with open(f"./result/{opt.exp_name}/log_filtered_index_list_{eval_dir}.txt", "a", encoding="utf-8") as f:
                        for e in dataset.filtered_index_list:
                            f.write(f"{e}\n")
    print(f"🗂️ Dataset List Contents: {dataset_list}")
    concatenated_dataset = ConcatDataset(dataset_list)

    return concatenated_dataset, dataset_log


class LmdbDataset(Dataset):

    def __init__(self, root, opt):

        self.root = root
        self.opt = opt
        self.env = lmdb.open(root, max_readers=32, readonly=True, lock=False, readahead=False, meminit=False)
        if not self.env:
            print('cannot create lmdb from %s' % (root))
            sys.exit(0)

        with self.env.begin(write=False) as txn:
            nSamples = int(txn.get('num-samples'.encode()))
            self.nSamples = nSamples

            if self.opt.data_filtering_off:
                # for fast check or benchmark evaluation with no filtering
                self.filtered_index_list = [index + 1 for index in range(self.nSamples)]
            else:
                """ Filtering part
                If you want to evaluate IC15-2077 & CUTE datasets which have special character labels,
                use --data_filtering_off and only evaluate on alphabets and digits.
                see https://github.com/clovaai/deep-text-recognition-benchmark/blob/6593928855fb7abb999a99f428b3e4477d4ae356/dataset.py#L190-L192

                And if you want to evaluate them with the model trained with --sensitive option,
                use --sensitive and --data_filtering_off,
                see https://github.com/clovaai/deep-text-recognition-benchmark/blob/dff844874dbe9e0ec8c5a52a7bd08c7f20afe704/test.py#L137-L144
                """
                self.filtered_index_list = []
                for index in range(self.nSamples):
                    index += 1  # lmdb starts with 1
                    label_key = 'label-%09d'.encode() % index
                    label = txn.get(label_key).decode('utf-8')

                    if len(label) > self.opt.batch_max_length:
                        # print(f'The length of the label is longer than max_length: length
                        # {len(label)}, {label} in dataset {self.root}')
                        continue

                    # By default, images containing characters which are not in opt.character are filtered.
                    # You can add [UNK] token to `opt.character` in utils.py instead of this filtering.
                    out_of_char = f'[^{self.opt.character}]'
                    if re.search(out_of_char, label.lower()):
                        continue

                    self.filtered_index_list.append(index)

                self.nSamples = len(self.filtered_index_list)

    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        assert index <= len(self), 'index range error'
        index = self.filtered_index_list[index]

        with self.env.begin(write=False) as txn:
            label_key = 'label-%09d'.encode() % index
            label = txn.get(label_key).decode('utf-8')
            img_key = 'image-%09d'.encode() % index
            imgbuf = txn.get(img_key)

            buf = six.BytesIO()
            buf.write(imgbuf)
            buf.seek(0)
            try:
                if self.opt.rgb:
                    img = Image.open(buf).convert('RGB')  # for color image
                else:
                    img = Image.open(buf).convert('L')

            except IOError:
                print(f'Corrupted image for {index}')
                # make dummy image and dummy label for corrupted image.
                if self.opt.rgb:
                    img = Image.new('RGB', (self.opt.imgW, self.opt.imgH))
                else:
                    img = Image.new('L', (self.opt.imgW, self.opt.imgH))
                label = '[dummy_label]'

            if not self.opt.sensitive:
                label = label.lower()

            # We only train and evaluate on alphanumerics (or pre-defined character set in train.py)
            out_of_char = f'[^{self.opt.character}]'
            label = re.sub(out_of_char, '', label)

        return (img, label)


class RawDataset(Dataset):

    def __init__(self, root, opt):
        self.opt = opt
        self.image_path_list = []
        for dirpath, dirnames, filenames in os.walk(root):
            for name in filenames:
                _, ext = os.path.splitext(name)
                ext = ext.lower()
                if ext == '.jpg' or ext == '.jpeg' or ext == '.png':
                    self.image_path_list.append(os.path.join(dirpath, name))

        self.image_path_list = natsorted(self.image_path_list)
        self.nSamples = len(self.image_path_list)

    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):

        try:
            if self.opt.rgb:
                img = Image.open(self.image_path_list[index]).convert('RGB')  # for color image
            else:
                img = Image.open(self.image_path_list[index]).convert('L')

        except IOError:
            print(f'Corrupted image for {index}')
            # make dummy image and dummy label for corrupted image.
            if self.opt.rgb:
                img = Image.new('RGB', (self.opt.imgW, self.opt.imgH))
            else:
                img = Image.new('L', (self.opt.imgW, self.opt.imgH))

        return (img, self.image_path_list[index])


class ResizeNormalize(object):

    def __init__(self, size, interpolation=Image.BICUBIC):
        self.size = size
        self.interpolation = interpolation
        self.toTensor = transforms.ToTensor()

    def __call__(self, img):
        img = img.resize(self.size, self.interpolation)
        img = self.toTensor(img)
        img.sub_(0.5).div_(0.5)
        return img


class NormalizePAD(object):

    def __init__(self, max_size, PAD_type='right'):
        self.toTensor = transforms.ToTensor()
        self.max_size = max_size
        self.max_width_half = math.floor(max_size[2] / 2)
        self.PAD_type = PAD_type

    def __call__(self, img):
        img = self.toTensor(img)
        img.sub_(0.5).div_(0.5)
        c, h, w = img.size()
        Pad_img = torch.FloatTensor(*self.max_size).fill_(0)
        Pad_img[:, :, :w] = img  # right pad
        if self.max_size[2] != w:  # add border Pad
            Pad_img[:, :, w:] = img[:, :, w - 1].unsqueeze(2).expand(c, h, self.max_size[2] - w)

        return Pad_img


class AlignCollate(object):

    def __init__(self, imgH=32, imgW=100, keep_ratio_with_pad=False):
        self.imgH = imgH
        self.imgW = imgW
        self.keep_ratio_with_pad = keep_ratio_with_pad

    def __call__(self, batch):
        batch = filter(lambda x: x is not None, batch)
        images, labels = zip(*batch)

        if self.keep_ratio_with_pad:  # same concept with 'Rosetta' paper
            resized_max_w = self.imgW
            input_channel = 3 if images[0].mode == 'RGB' else 1
            transform = NormalizePAD((input_channel, self.imgH, resized_max_w))

            resized_images = []
            for image in images:
                w, h = image.size
                ratio = w / float(h)
                if math.ceil(self.imgH * ratio) > self.imgW:
                    resized_w = self.imgW
                else:
                    resized_w = math.ceil(self.imgH * ratio)

                resized_image = image.resize((resized_w, self.imgH), Image.BICUBIC)
                resized_images.append(transform(resized_image))
                # resized_image.save('./image_test/%d_test.jpg' % w)

            image_tensors = torch.cat([t.unsqueeze(0) for t in resized_images], 0)

        else:
            transform = ResizeNormalize((self.imgW, self.imgH))
            image_tensors = [transform(image) for image in images]
            image_tensors = torch.cat([t.unsqueeze(0) for t in image_tensors], 0)

        return image_tensors, labels


def tensor2im(image_tensor, imtype=np.uint8):
    image_numpy = image_tensor.cpu().float().numpy()
    if image_numpy.shape[0] == 1:
        image_numpy = np.tile(image_numpy, (3, 1, 1))
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
    return image_numpy.astype(imtype)


def save_image(image_numpy, image_path):
    image_pil = Image.fromarray(image_numpy)
    image_pil.save(image_path)

#modifying the dataset.py to fit syntax of new pytorch version of next() instead of .next comment stops the cell from running properly


In [ ]:
%%writefile /kaggle/working/AttentionHTR/model/modules/feature_extraction.py


import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.vision_transformer import vit_b_16
from torchdiffeq import odeint_adjoint as odeint
from torchdiffeq import odeint
# ODE function definition
class ODEFunc(nn.Module):
    def __init__(self, input_dim):
        super(ODEFunc, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, input_dim)
    
    def forward(self, t, x):
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

# Neural ODE wrapper
# Neural ODE class to evolve the features over time
class NeuralODE(nn.Module):
    def __init__(self, ode_func, time_steps=2):
        super(NeuralODE, self).__init__()
        self.ode_func = ode_func
        self.time_steps = time_steps
    
    def forward(self, x):
        t = torch.linspace(0, 1, steps=self.time_steps, device=x.device, dtype=x.dtype)
        x_ode = odeint(self.ode_func, x, t, rtol=1e-1, atol=1e-1, options={"max_num_steps": 20})
        return x+x_ode[-1]

# Modified ResNet-style feature extractor with downsampling
class ResNet_FeatureExtractor_3layer(nn.Module):
    def __init__(self, input_channel, output_channel):
        super(ResNet_FeatureExtractor_3layer, self).__init__()
        
        self.relu = nn.ReLU(inplace=True)

        self.conv0_1 = nn.Conv2d(input_channel, output_channel // 2, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn0_1 = nn.BatchNorm2d(output_channel // 2)

        self.conv0_2 = nn.Conv2d(output_channel // 2, output_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn0_2 = nn.BatchNorm2d(output_channel)

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # ↓ by 2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # ↓ by 2 again

        # Final adaptive pooling to fixed size (e.g., 8x8)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((8, 8))  # Can use (14,14) or (7,7) based on memory

    def forward(self, x):
        x = self.relu(self.bn0_1(self.conv0_1(x)))  # [B, C//2, H, W]
        x = self.pool1(x)                           # ↓ H/2, W/2
        x = self.relu(self.bn0_2(self.conv0_2(x)))  # [B, C, H, W]
        x = self.pool2(x)                           # ↓ again
        x = self.adaptive_pool(x)                  # [B, C, 8, 8]
        return x

# Final combined class with Neural ODE
class ResNet_FeatureExtractor_With_NeuralODE(nn.Module):
    def __init__(self, input_channel, output_channel, time_steps=5):
        super(ResNet_FeatureExtractor_With_NeuralODE, self).__init__()
        self.feature_extractor = ResNet_FeatureExtractor_3layer(input_channel, output_channel)
        self.ode_func = ODEFunc(output_channel)
        self.neural_ode = NeuralODE(self.ode_func, time_steps)

    def forward(self, x):
        x = x.float()
        x = self.feature_extractor(x)  # (B, C, H, W), now H=W=8

        B, C, H, W = x.size()
        x = x.permute(0, 2, 3, 1).contiguous()  # [B, H, W, C]
        x = x.view(B, H * W, C)                 # [B, 64, C]

        x_ode = []
        for i in range(B):
            out = self.neural_ode(x[i])         # [64, C]
            x_ode.append(out)

        x_ode = torch.stack(x_ode, dim=0)       # [B, 64, C]

        return x_ode







class VGG_FeatureExtractor(nn.Module):
    """ FeatureExtractor of CRNN (https://arxiv.org/pdf/1507.05717.pdf) """

    def __init__(self, input_channel, output_channel=512):
        super(VGG_FeatureExtractor, self).__init__()
        self.output_channel = [int(output_channel / 8), int(output_channel / 4),
                               int(output_channel / 2), output_channel]  # [64, 128, 256, 512]
        self.ConvNet = nn.Sequential(
            nn.Conv2d(input_channel, self.output_channel[0], 3, 1, 1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # 64x16x50
            nn.Conv2d(self.output_channel[0], self.output_channel[1], 3, 1, 1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # 128x8x25
            nn.Conv2d(self.output_channel[1], self.output_channel[2], 3, 1, 1), nn.ReLU(True),  # 256x8x25
            nn.Conv2d(self.output_channel[2], self.output_channel[2], 3, 1, 1), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)),  # 256x4x25
            nn.Conv2d(self.output_channel[2], self.output_channel[3], 3, 1, 1, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True),  # 512x4x25
            nn.Conv2d(self.output_channel[3], self.output_channel[3], 3, 1, 1, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)),  # 512x2x25
            nn.Conv2d(self.output_channel[3], self.output_channel[3], 2, 1, 0), nn.ReLU(True))  # 512x1x24

    def forward(self, input):
        return self.ConvNet(input)


class RCNN_FeatureExtractor(nn.Module):
    """ FeatureExtractor of GRCNN (https://papers.nips.cc/paper/6637-gated-recurrent-convolution-neural-network-for-ocr.pdf) """

    def __init__(self, input_channel, output_channel=512):
        super(RCNN_FeatureExtractor, self).__init__()
        self.output_channel = [int(output_channel / 8), int(output_channel / 4),
                               int(output_channel / 2), output_channel]  # [64, 128, 256, 512]
        self.ConvNet = nn.Sequential(
            nn.Conv2d(input_channel, self.output_channel[0], 3, 1, 1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # 64 x 16 x 50
            GRCL(self.output_channel[0], self.output_channel[0], num_iteration=5, kernel_size=3, pad=1),
            nn.MaxPool2d(2, 2),  # 64 x 8 x 25
            GRCL(self.output_channel[0], self.output_channel[1], num_iteration=5, kernel_size=3, pad=1),
            nn.MaxPool2d(2, (2, 1), (0, 1)),  # 128 x 4 x 26
            GRCL(self.output_channel[1], self.output_channel[2], num_iteration=5, kernel_size=3, pad=1),
            nn.MaxPool2d(2, (2, 1), (0, 1)),  # 256 x 2 x 27
            nn.Conv2d(self.output_channel[2], self.output_channel[3], 2, 1, 0, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True))  # 512 x 1 x 26

    def forward(self, input):
        return self.ConvNet(input)


class ResNet_FeatureExtractor(nn.Module):
    """ FeatureExtractor of FAN (http://openaccess.thecvf.com/content_ICCV_2017/papers/Cheng_Focusing_Attention_Towards_ICCV_2017_paper.pdf) """

    def __init__(self, input_channel, output_channel=512):
        super(ResNet_FeatureExtractor, self).__init__()
        self.ConvNet = ResNet(input_channel, output_channel, BasicBlock, [1, 2, 5, 3])

    def forward(self, input):
        return self.ConvNet(input)


# For Gated RCNN
class GRCL(nn.Module):

    def __init__(self, input_channel, output_channel, num_iteration, kernel_size, pad):
        super(GRCL, self).__init__()
        self.wgf_u = nn.Conv2d(input_channel, output_channel, 1, 1, 0, bias=False)
        self.wgr_x = nn.Conv2d(output_channel, output_channel, 1, 1, 0, bias=False)
        self.wf_u = nn.Conv2d(input_channel, output_channel, kernel_size, 1, pad, bias=False)
        self.wr_x = nn.Conv2d(output_channel, output_channel, kernel_size, 1, pad, bias=False)

        self.BN_x_init = nn.BatchNorm2d(output_channel)

        self.num_iteration = num_iteration
        self.GRCL = [GRCL_unit(output_channel) for _ in range(num_iteration)]
        self.GRCL = nn.Sequential(*self.GRCL)

    def forward(self, input):
        """ The input of GRCL is consistant over time t, which is denoted by u(0)
        thus wgf_u / wf_u is also consistant over time t.
        """
        wgf_u = self.wgf_u(input)
        wf_u = self.wf_u(input)
        x = F.relu(self.BN_x_init(wf_u))

        for i in range(self.num_iteration):
            x = self.GRCL[i](wgf_u, self.wgr_x(x), wf_u, self.wr_x(x))

        return x


class GRCL_unit(nn.Module):

    def __init__(self, output_channel):
        super(GRCL_unit, self).__init__()
        self.BN_gfu = nn.BatchNorm2d(output_channel)
        self.BN_grx = nn.BatchNorm2d(output_channel)
        self.BN_fu = nn.BatchNorm2d(output_channel)
        self.BN_rx = nn.BatchNorm2d(output_channel)
        self.BN_Gx = nn.BatchNorm2d(output_channel)

    def forward(self, wgf_u, wgr_x, wf_u, wr_x):
        G_first_term = self.BN_gfu(wgf_u)
        G_second_term = self.BN_grx(wgr_x)
        G = F.sigmoid(G_first_term + G_second_term)

        x_first_term = self.BN_fu(wf_u)
        x_second_term = self.BN_Gx(self.BN_rx(wr_x) * G)
        x = F.relu(x_first_term + x_second_term)

        return x


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = self._conv3x3(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = self._conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def _conv3x3(self, in_planes, out_planes, stride=1):
        "3x3 convolution with padding"
        return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                         padding=1, bias=False)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, input_channel, output_channel, block, layers):
        super(ResNet, self).__init__()

        self.output_channel_block = [int(output_channel / 4), int(output_channel / 2), output_channel, output_channel]

        self.inplanes = int(output_channel / 8)
        self.conv0_1 = nn.Conv2d(input_channel, int(output_channel / 16),
                                 kernel_size=3, stride=1, padding=1, bias=False)
        self.bn0_1 = nn.BatchNorm2d(int(output_channel / 16))
        self.conv0_2 = nn.Conv2d(int(output_channel / 16), self.inplanes,
                                 kernel_size=3, stride=1, padding=1, bias=False)
        self.bn0_2 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)

        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.layer1 = self._make_layer(block, self.output_channel_block[0], layers[0])
        self.conv1 = nn.Conv2d(self.output_channel_block[0], self.output_channel_block[
                               0], kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.output_channel_block[0])

        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.layer2 = self._make_layer(block, self.output_channel_block[1], layers[1], stride=1)
        self.conv2 = nn.Conv2d(self.output_channel_block[1], self.output_channel_block[
                               1], kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(self.output_channel_block[1])

        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=(2, 1), padding=(0, 1))
        self.layer3 = self._make_layer(block, self.output_channel_block[2], layers[2], stride=1)
        self.conv3 = nn.Conv2d(self.output_channel_block[2], self.output_channel_block[
                               2], kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.output_channel_block[2])

        self.layer4 = self._make_layer(block, self.output_channel_block[3], layers[3], stride=1)
        self.conv4_1 = nn.Conv2d(self.output_channel_block[3], self.output_channel_block[
                                 3], kernel_size=2, stride=(2, 1), padding=(0, 1), bias=False)
        self.bn4_1 = nn.BatchNorm2d(self.output_channel_block[3])
        self.conv4_2 = nn.Conv2d(self.output_channel_block[3], self.output_channel_block[
                                 3], kernel_size=2, stride=1, padding=0, bias=False)
        self.bn4_2 = nn.BatchNorm2d(self.output_channel_block[3])

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv0_1(x)
        x = self.bn0_1(x)
        x = self.relu(x)
        x = self.conv0_2(x)
        x = self.bn0_2(x)
        x = self.relu(x)

        x = self.maxpool1(x)
        x = self.layer1(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.maxpool2(x)
        x = self.layer2(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.maxpool3(x)
        x = self.layer3(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.layer4(x)
        x = self.conv4_1(x)
        x = self.bn4_1(x)
        x = self.relu(x)
        x = self.conv4_2(x)
        x = self.bn4_2(x)
        x = self.relu(x)

        return x

class ViT_FeatureExtractor(nn.Module):
    """ Vision Transformer Feature Extractor (for Grayscale Images) """

    def __init__(self, input_channel=1, output_channel=768, image_size=128, patch_size=16):
        super(ViT_FeatureExtractor, self).__init__()

        # Initialize base ViT (without pretrained weights)
        self.vit = vit_b_16(pretrained=False, image_size=image_size)

        # Replace input conv layer for grayscale input
        if input_channel != 3:
            self.vit.conv_proj = nn.Conv2d(
                input_channel, 
                self.vit.conv_proj.out_channels, 
                kernel_size=patch_size, 
                stride=patch_size
            )

        # Dropout to prevent overfitting
        self.dropout = nn.Dropout(0.4)

        # Replace the classifier head with an Identity layer
        self.vit.heads = nn.Identity()

        # Project the output from the ViT encoder to the desired output channel
        self.projector = nn.Linear(self.vit.hidden_dim, output_channel)

    def forward(self, x):
        # vit returns shape [B, T+1, C], where T = num_patches
        tokens = self.vit._process_input(x)  # token embedding
        n, _, h, w = x.shape
        x = self.vit.conv_proj(x)  # [B, C, H', W']
        x = x.flatten(2).transpose(1, 2)  # [B, num_patches, C]
        class_token = self.vit.class_token.expand(n, -1, -1)
        x = torch.cat((class_token, x), dim=1)
        x = x + self.vit.encoder.pos_embedding
        x = self.vit.encoder.dropout(x)
        x = self.vit.encoder.layers(x)
        return x[:, 1:, :]  # remove CLS token, return [B, T, C]

In [ ]:
%%writefile /kaggle/working/AttentionHTR/model/model.py
from torchvision.models.vision_transformer import vit_b_16
import torch.nn as nn

from modules.transformation import TPS_SpatialTransformerNetwork
from modules.feature_extraction import NeuralODE,ResNet_FeatureExtractor_With_NeuralODE,ResNet_FeatureExtractor_3layer,ViT_FeatureExtractor,VGG_FeatureExtractor, RCNN_FeatureExtractor, ResNet_FeatureExtractor
from modules.sequence_modeling import BidirectionalLSTM
from modules.prediction import Attention


class Model(nn.Module):

    def __init__(self, opt):
        super(Model, self).__init__()
        self.opt = opt
        self.stages = {'Trans': opt.Transformation, 'Feat': opt.FeatureExtraction,
                       'Seq': opt.SequenceModeling, 'Pred': opt.Prediction}

        """ Transformation """
        if opt.Transformation == 'TPS':
            self.Transformation = TPS_SpatialTransformerNetwork(
                F=opt.num_fiducial, I_size=(opt.imgH, opt.imgW), I_r_size=(opt.imgH, opt.imgW), I_channel_num=opt.input_channel)
        else:
            print('No Transformation module specified')

        """ FeatureExtraction """
        if opt.FeatureExtraction == 'VGG':
            self.FeatureExtraction = VGG_FeatureExtractor(opt.input_channel, opt.output_channel)
        elif opt.FeatureExtraction == 'RCNN':
            self.FeatureExtraction = RCNN_FeatureExtractor(opt.input_channel, opt.output_channel)
        elif opt.FeatureExtraction == 'ResNet':
            self.FeatureExtraction = ResNet_FeatureExtractor(opt.input_channel, opt.output_channel)
        elif opt.FeatureExtraction == 'ViT':
            self.FeatureExtraction = ViT_FeatureExtractor(opt.input_channel, opt.output_channel)
        elif opt.FeatureExtraction == 'neural':
            self.FeatureExtraction = ResNet_FeatureExtractor_With_NeuralODE(opt.input_channel, opt.output_channel)
        else:
            raise Exception('No FeatureExtraction module specified')
        self.FeatureExtraction_output = opt.output_channel  # int(imgH/16-1) * 512
        self.AdaptiveAvgPool = nn.AdaptiveAvgPool2d((None, 1))  # Transform final (imgH/16-1) -> 1

        """ Sequence modeling"""
        if opt.SequenceModeling == 'BiLSTM':
            self.SequenceModeling = nn.Sequential(
                BidirectionalLSTM(self.FeatureExtraction_output, opt.hidden_size, opt.hidden_size),
                BidirectionalLSTM(opt.hidden_size, opt.hidden_size, opt.hidden_size))
            self.SequenceModeling_output = opt.hidden_size
        else:
            print('No SequenceModeling module specified')
            self.SequenceModeling_output = self.FeatureExtraction_output
        

        """ Prediction """
        if opt.Prediction == 'CTC':
            self.Prediction = nn.Linear(self.SequenceModeling_output, opt.num_class)
        elif opt.Prediction == 'Attn':
            self.Prediction = Attention(self.SequenceModeling_output, opt.hidden_size, opt.num_class)
        else:
            raise Exception('Prediction is neither CTC or Attn')
              
    def forward(self, input, text, is_train=True):
        """ Transformation stage """
        if not self.stages['Trans'] == "None":
            input = self.Transformation(input)
    
        """ Feature extraction stage """
        visual_feature = self.FeatureExtraction(input)

        if self.stages['Feat'] == 'ViT':
            #print("ViT raw output shape:", visual_feature.shape)
            if visual_feature.dim() == 4:
                B, C, H, W = visual_feature.size()
                visual_feature = visual_feature.permute(0, 2, 3, 1).reshape(B, H * W, C)
            elif visual_feature.dim() == 3:
                # [B, T, C] → correct shape already
                pass
            else:
                raise ValueError(f"Unexpected ViT output shape: {visual_feature.shape}")

    
        """ Sequence modeling stage """
        if self.stages['Seq'] == 'BiLSTM':
            contextual_feature = self.SequenceModeling(visual_feature)  # [B, T, C]
            #print("contextual_feature (batch_H) shape:", contextual_feature.shape)
            #print("text shape:", text.shape)
        else:
            contextual_feature = visual_feature  # If no RNN stage is selected

        #print("Encoder output shape:", visual_feature.shape)  # This is batch_H

    
        """ Prediction stage """
        prediction = self.Prediction(
            contextual_feature, text, is_train, batch_max_length=self.opt.batch_max_length
        )
        return prediction
    

    


In [ ]:
%%writefile /kaggle/working/AttentionHTR/model/train.py
import os
import sys
import time
import random
import string
import argparse

import torch
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import numpy as np
import matplotlib.pyplot as plt

from utils import CTCLabelConverter, CTCLabelConverterForBaiduWarpctc, AttnLabelConverter, Averager
from dataset import hierarchical_dataset, AlignCollate, Batch_Balanced_Dataset
from model import Model
from test import validation
from pytorchtools import EarlyStopping
import torch.nn.functional as F

def label_smoothing_loss(preds, targets, smoothing=0.1, ignore_index=0):
    """
    preds: [batch_size, seq_len, num_classes]
    targets: [batch_size, seq_len]
    """
    n_class = preds.size(-1)
    confidence = 1.0 - smoothing
    smoothed_label = torch.full_like(preds, smoothing / (n_class - 1))
    smoothed_label.scatter_(2, targets.unsqueeze(2), confidence)
    smoothed_label[:, :, ignore_index] = 0  # Optional: ignore index
    loss = -torch.sum(F.log_softmax(preds, dim=2) * smoothed_label, dim=2)

    # mask out ignored targets
    mask = targets.ne(ignore_index)
    loss = loss * mask.float()
    return loss.sum() / mask.sum()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def train(opt):
    """ dataset preparation """
    if not opt.data_filtering_off:
        print('Filtering the images containing characters which are not in opt.character')
        print('Filtering the images whose label is longer than opt.batch_max_length')
        # see https://github.com/clovaai/deep-text-recognition-benchmark/blob/6593928855fb7abb999a99f428b3e4477d4ae356/dataset.py#L130

    opt.select_data = opt.select_data.split('-')
    opt.batch_ratio = opt.batch_ratio.split('-')
    train_dataset = Batch_Balanced_Dataset(opt)

    log = open(f'./saved_models/{opt.exp_name}/log_dataset.txt', 'a')
    AlignCollate_valid = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
    valid_dataset, valid_dataset_log = hierarchical_dataset(root=opt.valid_data, opt=opt)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=opt.batch_size,
        shuffle=True,  # 'True' to check training progress with validation function.
        num_workers=int(opt.workers),
        collate_fn=AlignCollate_valid, pin_memory=True)
    log.write(valid_dataset_log)
    print('-' * 80)
    log.write('-' * 80 + '\n')
    log.close()
    
    """ model configuration """
    if 'CTC' in opt.Prediction:
        if opt.baiduCTC:
            converter = CTCLabelConverterForBaiduWarpctc(opt.character)
        else:
            converter = CTCLabelConverter(opt.character)
    else:
        converter = AttnLabelConverter(opt.character)
    opt.num_class = len(converter.character)

    if opt.rgb:
        opt.input_channel = 3
    model = Model(opt)
    print('model input parameters', opt.imgH, opt.imgW, opt.num_fiducial, opt.input_channel, opt.output_channel,
          opt.hidden_size, opt.num_class, opt.batch_max_length, opt.Transformation, opt.FeatureExtraction,
          opt.SequenceModeling, opt.Prediction)

    # weight initialization
    for name, param in model.named_parameters():
        if 'localization_fc2' in name:
            print(f'Skip {name} as it is already initialized')
            continue
        try:
            if 'bias' in name:
                init.constant_(param, 0.0)
            elif 'weight' in name:
                init.kaiming_normal_(param)
        except Exception as e:  # for batchnorm.
            if 'weight' in name:
                param.data.fill_(1)
            continue

    # data parallel for multi-GPU
    model = torch.nn.DataParallel(model).to(device)
    model.train()
    if opt.saved_model != '':
        print(f'loading pretrained model from {opt.saved_model}')
        if opt.FT:
            model.load_state_dict(torch.load(opt.saved_model), strict=False)
        else:
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            start_epoch = checkpoint['epoch'] + 1
            loss = checkpoint['loss']
    print("Model:")
    print(model)

    """ setup loss """
    if 'CTC' in opt.Prediction:
        if opt.baiduCTC:
            # need to install warpctc. see our guideline.
            from warpctc_pytorch import CTCLoss 
            criterion = CTCLoss()
        else:
            criterion = torch.nn.CTCLoss(zero_infinity=True).to(device)
    else:
        criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)  # ignore [GO] token = ignore index 0
    # loss averager
    loss_avg = Averager()

    # filter that only require gradient decent
    filtered_parameters = []
    params_num = []
    for p in filter(lambda p: p.requires_grad, model.parameters()):
        filtered_parameters.append(p)
        params_num.append(np.prod(p.size()))
    print('Trainable params num : ', sum(params_num))
    # [print(name, p.numel()) for name, p in filter(lambda p: p[1].requires_grad, model.named_parameters())]

    # setup optimizer
    if opt.adam:
        optimizer = optim.Adam(filtered_parameters, lr=opt.lr, betas=(opt.beta1, 0.999))
    else:
        optimizer = optim.Adadelta(filtered_parameters, lr=opt.lr, rho=opt.rho, eps=opt.eps)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    print("Optimizer:")
    print(optimizer)

    """ final options """
    # print(opt)
    with open(f'./saved_models/{opt.exp_name}/opt.txt', 'a') as opt_file:
        opt_log = '------------ Options -------------\n'
        args = vars(opt)
        for k, v in args.items():
            opt_log += f'{str(k)}: {str(v)}\n'
        opt_log += '---------------------------------------\n'
        print(opt_log)
        opt_file.write(opt_log)

    """ start training """
    start_iter = 0
    if opt.saved_model != '':
        try:
            start_iter = int(opt.saved_model.split('_')[-1].split('.')[0])
            print(f'continue to train, start_iter: {start_iter}')
        except:
            pass

    start_time = time.time()
    best_accuracy = -1
    best_norm_ED = -1
    iteration = start_iter

    # ----------------------------------------------------------------------------------------
    
    # Initialize the early_stopping object
    early_stopping = EarlyStopping(patience=opt.patience, verbose=True, path=f'./saved_models/{opt.exp_name}/checkpoint-seed{opt.manualSeed}.pth')
    
    # Debug information.
    n_train_samples = len(train_dataset.data_loader_list[0].dataset)
    p1 = f"Number of training samples: {n_train_samples}"
    iter_per_epoch = n_train_samples // opt.batch_size
    n_epochs = opt.num_iter // iter_per_epoch
    p2 = f"Number of epochs: {n_epochs}, iter per epoch: {iter_per_epoch}"
    epoch = 0
    train_losses = []
    valid_losses = []
    accuracies = []
    epochs_list = []
    norm_ed_list=[]
    # Start logging early stopping information.
    valid_log_fname = f'./saved_models/{opt.exp_name}/log_early.txt'
    early_stopping.log(p1 + "\n" + p2 + "\n", valid_log_fname)
    
    # ----------------------------------------------------------------------------------------

    while(True):
        
        # train part
        #print("Fetching batch...")
        image_tensors, labels = train_dataset.get_batch()
        #print("Fetched batch:", image_tensors.shape, len(labels))  # optional
        image = image_tensors.to(device)

        #print("Running forward pass...")
        try:
            text, length = converter.encode(labels, batch_max_length=opt.batch_max_length)
            if 'CTC' in opt.Prediction:
                preds = model(image, text)
            else:
                preds = model(image, text[:, :-1])  # align with Attention.forward
            # print("Forward pass successful. Output shape:", preds.shape)
        except Exception as e:
            print("Error during forward pass:", str(e))

        batch_size = image.size(0)

        if 'CTC' in opt.Prediction:
            preds = model(image, text)
            preds_size = torch.IntTensor([preds.size(1)] * batch_size)
            if opt.baiduCTC:
                preds = preds.permute(1, 0, 2)  # to use CTCLoss format
                cost = criterion(preds, text, preds_size, length) / batch_size
            else:
                preds = preds.log_softmax(2).permute(1, 0, 2)
                cost = criterion(preds, text, preds_size, length)

        else:
            preds = model(image, text[:, :-1])  # align with Attention.forward
            target = text[:, 1:]  # without [GO] Symbol
            cost=criterion(preds.view(-1, preds.shape[-1]), target.contiguous().view(-1))
            #cost = label_smoothing_loss(preds, target, smoothing=0.1)

        model.zero_grad()
        torch.cuda.empty_cache()
        cost.backward()
        # for param in model.parameters():
        #     if param.grad is not None:
        #         print(f"{param.name} gradient mean: {param.grad.mean()}")
        torch.nn.utils.clip_grad_norm_(model.parameters(), opt.grad_clip)  # gradient clipping with 5 (Default)
        optimizer.step()

        loss_avg.add(cost)

        # validation part
        print(f"iteration {iteration}")
        if (iteration + 1) % opt.valInterval == 0 or iteration == 0:  # To see training progress, we also conduct validation when 'iteration == 0'
            elapsed_time = time.time() - start_time
            # for log
            with open(f'./saved_models/{opt.exp_name}/log_train.txt', 'a') as log:
                model.eval()
                with torch.no_grad():
                    # valid_iter = iter(valid_loader)
                    # dummy_image_tensors, dummy_labels = next(valid_iter)
                    # dummy_image = dummy_image_tensors.to(device)
                    # dummy_text, dummy_length = converter.encode(dummy_labels, batch_max_length=opt.batch_max_length)

                    # print("Running validation forward pass on dummy batch")
                    # if 'CTC' in opt.Prediction:
                    #     dummy_preds = model(dummy_image, dummy_text)
                    #     print("Validation preds shape (CTC):", dummy_preds.shape)
                    # else:
                    #     dummy_preds = model(dummy_image, dummy_text[:, :-1])
                    #     print("Validation preds shape (Attention):", dummy_preds.shape)
                    #     # OPTIONAL: check for NaN or inf values
                    # if torch.isnan(dummy_preds).any():
                    #     print("WARNING: dummy_preds contains NaN values!")
                    # if torch.isinf(dummy_preds).any():
                    #     print("WARNING: dummy_preds contains inf values!")
                        # Run full validation
                    valid_loss, current_accuracy, current_norm_ED, preds, confidence_score, labels, infer_time, length_of_data = validation(
                    model, criterion, valid_loader, converter, opt)
                    #print("Successfully skipped full validation step — no freeze here")
                model.train()

                # training loss and validation loss
                train_loss = loss_avg.val()
                loss_log = f'[{iteration+1}/{opt.num_iter}] Train loss: {train_loss:0.5f}, Valid loss: {valid_loss:0.5f}, Elapsed_time: {elapsed_time:0.5f}'
                loss_avg.reset()

                current_model_log = f'{"Current_accuracy":17s}: {current_accuracy:0.3f}, {"Current_norm_ED":17s}: {current_norm_ED:0.2f}'
                #schedule learning rate according to val loss improving or not after 5 epochs if not improved after 5 epochs cut down lr by 0.5
                scheduler.step(valid_loss)
                # keep best accuracy model (on valid dataset)
                if current_accuracy > best_accuracy:
                    best_accuracy = current_accuracy
                    checkpoint = {
                    'epoch': iteration,  # or your epoch number
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),  # if using scheduler
                    'best_accuracy': best_accuracy,
                    'best_norm_ED': best_norm_ED,
                    'loss': valid_loss
                   }
                    torch.save(checkpoint, f'./saved_models/{opt.exp_name}/checkpoint_best_accuracy.pth')
                    #torch.save(model.state_dict(), f'./saved_models/{opt.exp_name}/best_accuracy.pth')
                if current_norm_ED > best_norm_ED:
                    best_norm_ED = current_norm_ED
                    checkpoint = {
                    'epoch': iteration,  # or your epoch number
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),  # if using scheduler
                    'best_accuracy': best_accuracy,
                    'best_norm_ED': best_norm_ED,
                    'loss': valid_loss
                   }
                    torch.save(checkpoint, f'./saved_models/{opt.exp_name}/checkpoint_best_norm_ED.pth')
                    #torch.save(model.state_dict(), f'./saved_models/{opt.exp_name}/best_norm_ED.pth')
                best_model_log = f'{"Best_accuracy":17s}: {best_accuracy:0.3f}, {"Best_norm_ED":17s}: {best_norm_ED:0.2f}'

                loss_model_log = f'{loss_log}\n{current_model_log}\n{best_model_log}'
                print(loss_model_log)
                log.write(loss_model_log + '\n')

                # show some predicted results
                dashed_line = '-' * 80
                head = f'{"Ground Truth":25s} | {"Prediction":25s} | Confidence Score & T/F'
                predicted_result_log = f'{dashed_line}\n{head}\n{dashed_line}\n'
                for gt, pred, confidence in zip(labels[:5], preds[:5], confidence_score[:5]):
                    if 'Attn' in opt.Prediction:
                        gt = gt[:gt.find('[s]')]
                        pred = pred[:pred.find('[s]')]

                    predicted_result_log += f'{gt:25s} | {pred:25s} | {confidence:0.4f}\t{str(pred == gt)}\n'
                predicted_result_log += f'{dashed_line}'
                print(predicted_result_log)
                log.write(predicted_result_log + '\n')
            
            # Check early stopping at each epoch.
            if (iteration + 1) % iter_per_epoch == 0 or iteration == 0:
                
                valid_log = f"Iter: [{iteration+1}/{opt.num_iter}]. Epoch: [{epoch}/{n_epochs}]. Training loss: {train_loss}."
              
                # early_stopping needs the validation loss to check if it has decreased, 
                # and if it has, it will make a checkpoint of the current model
                early_stopping(valid_loss, model, valid_log, valid_log_fname)
                if early_stopping.early_stop:
                    print("Early stopping")
                    early_stopping.log("Early stopping", valid_log_fname)
                    sys.exit() 
                train_losses.append(train_loss)
                valid_losses.append(valid_loss)
                accuracies.append(current_accuracy)
                epochs_list.append(epoch)
                norm_ed_list.append(current_norm_ED)
                
                if (epoch % 2 == 0) and epoch != 0:             
                    plt.figure(figsize=(12,4))
                    
                    plt.subplot(1,2,1)
                    plt.plot(epochs_list, train_losses, label='Train Loss')
                    plt.plot(epochs_list, valid_losses, label='Valid Loss')
                    plt.xlabel('Epoch')
                    plt.ylabel('Loss')
                    plt.title('Losses vs Epochs')
                    plt.legend()

                    plt.subplot(1,2,2)
                    plt.plot(epochs_list, norm_ed_list, label='Norm_ED')
                    plt.xlabel('Epoch')
                    plt.ylabel('Norm_ED')
                    plt.title('Norm_ED vs Epoch')
                    plt.legend()
                
                    plt.subplot(1,2,2)
                    plt.plot(epochs_list, accuracies, label='Accuracy')
                    plt.xlabel('Epoch')
                    plt.ylabel('Accuracy (%)')
                    plt.title('Accuracy vs Epoch')
                    plt.legend()
                
                    plt.suptitle(f'Performance at Epoch {epoch}')
                    plt.tight_layout()
                    plt.show()
                epoch += 1
        # save model per 1e+5 iter.
        if (iteration + 1) % (int(1e5 / 2)) == 0:
            torch.save(
                model.state_dict(), f'./saved_models/{opt.exp_name}/iter_{iteration+1}.pth')

        if (iteration + 1) == opt.num_iter:
            print('end the training')
            sys.exit()
        iteration += 1


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--exp_name', help='Where to store logs and models')
    parser.add_argument('--train_data', required=True, help='path to training dataset')
    parser.add_argument('--valid_data', required=True, help='path to validation dataset')
    parser.add_argument('--manualSeed', type=int, default=1111, help='for random seed setting')
    parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
    parser.add_argument('--batch_size', type=int, default=192, help='input batch size')
    parser.add_argument('--num_iter', type=int, default=300000, help='number of iterations to train for')
    parser.add_argument('--valInterval', type=int, default=2000, help='Interval between each validation')
    parser.add_argument('--saved_model', default='', help="path to model to continue training")
    parser.add_argument('--FT', action='store_true', help='whether to do fine-tuning')
    parser.add_argument('--adam', action='store_true', help='Whether to use adam (default is Adadelta)')
    parser.add_argument('--lr', type=float, default=1, help='learning rate, default=1.0 for Adadelta')
    parser.add_argument('--beta1', type=float, default=0.9, help='beta1 for adam. default=0.9')
    parser.add_argument('--rho', type=float, default=0.95, help='decay rate rho for Adadelta. default=0.95')
    parser.add_argument('--eps', type=float, default=1e-8, help='eps for Adadelta. default=1e-8')
    parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping value. default=5')
    parser.add_argument('--baiduCTC', action='store_true', help='for data_filtering_off mode')
    """ Data processing """
    parser.add_argument('--select_data', type=str, default='MJ-ST',
                        help='select training data (default is MJ-ST, which means MJ and ST used as training data)')
    parser.add_argument('--batch_ratio', type=str, default='0.5-0.5',
                        help='assign ratio for each selected data in the batch')
    parser.add_argument('--total_data_usage_ratio', type=str, default='1.0',
                        help='total data usage ratio, this ratio is multiplied to total number of data.')
    parser.add_argument('--batch_max_length', type=int, default=25, help='maximum-label-length')
    parser.add_argument('--imgH', type=int, default=32, help='the height of the input image')
    parser.add_argument('--imgW', type=int, default=100, help='the width of the input image')
    parser.add_argument('--rgb', action='store_true', help='use rgb input')
    parser.add_argument('--character', type=str,
                        default='0123456789abcdefghijklmnopqrstuvwxyz', help='character label')
    parser.add_argument('--sensitive', action='store_true', help='for sensitive character mode')
    parser.add_argument('--PAD', action='store_true', help='whether to keep ratio then pad for image resize')
    parser.add_argument('--data_filtering_off', action='store_true', help='for data_filtering_off mode')
    """ Model Architecture """
    parser.add_argument('--Transformation', type=str, required=True, help='Transformation stage. None|TPS')
    parser.add_argument('--FeatureExtraction', type=str, required=True,
                        help='FeatureExtraction stage. VGG|RCNN|ResNet')
    parser.add_argument('--SequenceModeling', type=str, required=True, help='SequenceModeling stage. None|BiLSTM')
    parser.add_argument('--Prediction', type=str, required=True, help='Prediction stage. CTC|Attn')
    parser.add_argument('--num_fiducial', type=int, default=20, help='number of fiducial points of TPS-STN')
    parser.add_argument('--input_channel', type=int, default=1,
                        help='the number of input channel of Feature extractor')
    parser.add_argument('--output_channel', type=int, default=512,
                        help='the number of output channel of Feature extractor')
    parser.add_argument('--hidden_size', type=int, default=256, help='the size of the LSTM hidden state')
    parser.add_argument('--patience', type=int, default=5, help='patience for the early stopping')

    opt = parser.parse_args()

    if not opt.exp_name:
        opt.exp_name = f'{opt.Transformation}-{opt.FeatureExtraction}-{opt.SequenceModeling}-{opt.Prediction}'
        opt.exp_name += f'-Seed{opt.manualSeed}'
        # print(opt.exp_name)

    os.makedirs(f'./saved_models/{opt.exp_name}', exist_ok=True)

    """ vocab / character number configuration """
    if opt.sensitive:
        # opt.character += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

    """ Seed and GPU setting """
    # print("Random Seed: ", opt.manualSeed)
    random.seed(opt.manualSeed)
    np.random.seed(opt.manualSeed)
    torch.manual_seed(opt.manualSeed)
    torch.cuda.manual_seed(opt.manualSeed)

    cudnn.benchmark = True
    cudnn.deterministic = True
    opt.num_gpu = torch.cuda.device_count()
    # print('device count', opt.num_gpu)
    if opt.num_gpu > 1:
        print('------ Use multi-GPU setting ------')
        print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
        # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
        opt.workers = opt.workers * opt.num_gpu
        opt.batch_size = opt.batch_size * opt.num_gpu

        """ previous version
        print('To equlize batch stats to 1-GPU setting, the batch_size is multiplied with num_gpu and multiplied batch_size is ', opt.batch_size)
        opt.batch_size = opt.batch_size * opt.num_gpu
        print('To equalize the number of epochs to 1-GPU setting, num_iter is divided with num_gpu by default.')
        If you dont care about it, just commnet out these line.)
        opt.num_iter = int(opt.num_iter / opt.num_gpu)
        """

    train(opt)

In [ ]:
%%writefile /kaggle/working/AttentionHTR/model/test.py
""" a modified version of deep-text-recognition-benchmark repository https://github.com/clovaai/deep-text-recognition-benchmark/blob/master/test.py """

import os
import time
import string
import argparse
import re

import torch
import torch.backends.cudnn as cudnn
import torch.utils.data
import torch.nn.functional as F
import numpy as np
from nltk.metrics.distance import edit_distance

from utils import CTCLabelConverter, AttnLabelConverter, Averager
from dataset import hierarchical_dataset, AlignCollate
from model import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def benchmark_all_eval(model, criterion, converter, opt, calculate_infer_time=False):
    """ evaluation with 10 benchmark evaluation datasets """
    # The evaluation datasets, dataset order is same with Table 1 in our paper.
    eval_data_list = ['IIIT5k_3000', 'SVT', 'IC03_860', 'IC03_867', 'IC13_857',
                      'IC13_1015', 'IC15_1811', 'IC15_2077', 'SVTP', 'CUTE80']

    # # To easily compute the total accuracy of our paper.
    # eval_data_list = ['IIIT5k_3000', 'SVT', 'IC03_867', 
    #                   'IC13_1015', 'IC15_2077', 'SVTP', 'CUTE80']

    if calculate_infer_time:
        evaluation_batch_size = 1  # batch_size should be 1 to calculate the GPU inference time per image.
    else:
        evaluation_batch_size = opt.batch_size

    list_accuracy = []
    total_forward_time = 0
    total_evaluation_data_number = 0
    total_correct_number = 0
    log = open(f'./result/{opt.exp_name}/log_all_evaluation.txt', 'a')
    dashed_line = '-' * 80
    print(dashed_line)
    log.write(dashed_line + '\n')
    for eval_data in eval_data_list:
        eval_data_path = os.path.join(opt.eval_data, eval_data)
        AlignCollate_evaluation = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
        eval_data, eval_data_log = hierarchical_dataset(root=eval_data_path, opt=opt)
        evaluation_loader = torch.utils.data.DataLoader(
            eval_data, batch_size=evaluation_batch_size,
            shuffle=False,
            num_workers=int(opt.workers),
            collate_fn=AlignCollate_evaluation, pin_memory=True)

        _, accuracy_by_best_model, norm_ED_by_best_model, _, _, _, infer_time, length_of_data = validation(
            model, criterion, evaluation_loader, converter, opt)
        list_accuracy.append(f'{accuracy_by_best_model:0.3f}')
        total_forward_time += infer_time
        total_evaluation_data_number += len(eval_data)
        total_correct_number += accuracy_by_best_model * length_of_data
        log.write(eval_data_log)
        print(f'Acc {accuracy_by_best_model:0.3f}\t normalized_ED {norm_ED_by_best_model:0.3f}')
        log.write(f'Acc {accuracy_by_best_model:0.3f}\t normalized_ED {norm_ED_by_best_model:0.3f}\n')
        print(dashed_line)
        log.write(dashed_line + '\n')

    averaged_forward_time = total_forward_time / total_evaluation_data_number * 1000
    total_accuracy = total_correct_number / total_evaluation_data_number
    params_num = sum([np.prod(p.size()) for p in model.parameters()])

    evaluation_log = 'accuracy: '
    for name, accuracy in zip(eval_data_list, list_accuracy):
        evaluation_log += f'{name}: {accuracy}\t'
    evaluation_log += f'total_accuracy: {total_accuracy:0.3f}\t'
    evaluation_log += f'averaged_infer_time: {averaged_forward_time:0.3f}\t# parameters: {params_num/1e6:0.3f}'
    print(evaluation_log)
    log.write(evaluation_log + '\n')
    log.close()

    return None


def validation(model, criterion, evaluation_loader, converter, opt):
    """ validation or evaluation """
    n_correct = 0
    norm_ED = 0
    length_of_data = 0
    infer_time = 0
    valid_loss_avg = Averager()

    # Export predictions only when testing
    # because dir `/result/{opt.exp_name}` is created only during testing,
    # but this function is also be called by train.py.
    # Log file with predictions contains the dir of test data.
    if hasattr(opt, 'eval_data'):
        eval_dir = opt.eval_data.split("/")[-1]
        log_predictions = open(f'./result/{opt.exp_name}/log_predictions_{eval_dir}.txt', 'a')
        log_predictions.write(f'batch,target,prediction,match,cum_match\n')

    for i, (image_tensors, labels) in enumerate(evaluation_loader):
        batch_size = image_tensors.size(0)
        length_of_data = length_of_data + batch_size
        image = image_tensors.to(device)
        # For max length prediction
        length_for_pred = torch.IntTensor([opt.batch_max_length] * batch_size).to(device)
        text_for_pred = torch.LongTensor(batch_size, opt.batch_max_length + 1).fill_(0).to(device)

        text_for_loss, length_for_loss = converter.encode(labels, batch_max_length=opt.batch_max_length)

        start_time = time.time()
        if 'CTC' in opt.Prediction:
            preds = model(image, text_for_pred)
            forward_time = time.time() - start_time

            # Calculate evaluation loss for CTC deocder.
            preds_size = torch.IntTensor([preds.size(1)] * batch_size)
            # permute 'preds' to use CTCloss format
            if opt.baiduCTC:
                cost = criterion(preds.permute(1, 0, 2), text_for_loss, preds_size, length_for_loss) / batch_size
            else:
                cost = criterion(preds.log_softmax(2).permute(1, 0, 2), text_for_loss, preds_size, length_for_loss)

            # Select max probabilty (greedy decoding) then decode index to character
            if opt.baiduCTC:
                _, preds_index = preds.max(2)
                preds_index = preds_index.view(-1)
            else:
                _, preds_index = preds.max(2)
            preds_str = converter.decode(preds_index.data, preds_size.data)
        
        else:
            preds = model(image, text_for_pred, is_train=False)
            forward_time = time.time() - start_time

            preds = preds[:, :text_for_loss.shape[1] - 1, :]
            target = text_for_loss[:, 1:]  # without [GO] Symbol
            cost = criterion(preds.contiguous().view(-1, preds.shape[-1]), target.contiguous().view(-1))

            # select max probabilty (greedy decoding) then decode index to character
            _, preds_index = preds.max(2)
            preds_str = converter.decode(preds_index, length_for_pred)
            labels = converter.decode(text_for_loss[:, 1:], length_for_loss)

        infer_time += forward_time
        valid_loss_avg.add(cost)

        # calculate accuracy & confidence score
        #print(f"preds BEFORE clamp min: {preds.min().item()}, max: {preds.max().item()}")
        preds = torch.clamp(preds, min=-10, max=10)
        #print(f"preds AFTER clamp min: {preds.min().item()}, max: {preds.max().item()}")
        preds_prob = F.softmax(preds, dim=2)
        preds_max_prob, _ = preds_prob.max(dim=2)
        confidence_score_list = []
        for gt, pred, pred_max_prob in zip(labels, preds_str, preds_max_prob):
            if 'Attn' in opt.Prediction:
                gt = gt[:gt.find('[s]')]
                pred_EOS = pred.find('[s]')
                pred = pred[:pred_EOS]  # prune after "end of sentence" token ([s])
                pred_max_prob = pred_max_prob[:pred_EOS]

            # To evaluate 'case sensitive model' with alphanumeric and case insensitve setting.
            if opt.sensitive and opt.data_filtering_off:
                pred = pred.lower()
                gt = gt.lower()
                alphanumeric_case_insensitve = '0123456789abcdefghijklmnopqrstuvwxyz'
                out_of_alphanumeric_case_insensitve = f'[^{alphanumeric_case_insensitve}]'
                pred = re.sub(out_of_alphanumeric_case_insensitve, '', pred)
                gt = re.sub(out_of_alphanumeric_case_insensitve, '', gt)

            if pred == gt:
                n_correct += 1

            # Export predictions only when testing. This function is also be called by train.py.
            if hasattr(opt, 'eval_data'):
                log_predictions.write(f'{i},{gt},{pred},{int(pred == gt)},{n_correct}\n')

            '''
            (old version) ICDAR2017 DOST Normalized Edit Distance https://rrc.cvc.uab.es/?ch=7&com=tasks
            "For each word we calculate the normalized edit distance to the length of the ground truth transcription."
            if len(gt) == 0:
                norm_ED += 1
            else:
                norm_ED += edit_distance(pred, gt) / len(gt)
            '''

            # ICDAR2019 Normalized Edit Distance
            if len(gt) == 0 or len(pred) == 0:
                norm_ED += 0
            elif len(gt) > len(pred):
                norm_ED += edit_distance(pred, gt) / len(gt)
            else:
                norm_ED += edit_distance(pred, gt) / len(gt) # Changed to gt from pred.

            # calculate confidence score (= multiply of pred_max_prob)
            try:
                confidence_score = pred_max_prob.cumprod(dim=0)[-1]
            except:
                confidence_score = 0  # for empty pred case, when prune after "end of sentence" token ([s])
            confidence_score_list.append(confidence_score)
            # print(pred, gt, pred==gt, confidence_score)

    accuracy = n_correct / float(length_of_data) * 100
    norm_ED = norm_ED / float(length_of_data)  # ICDAR2019 Normalized Edit Distance https://arxiv.org/pdf/1909.07741.pdf

    # Export predictions only when testing. This function is also be called by train.py.
    if hasattr(opt, 'eval_data'):
        log_predictions.close()

    return valid_loss_avg.val(), accuracy, norm_ED, preds_str, confidence_score_list, labels, infer_time, length_of_data


def test(opt):
    """ model configuration """
    if 'CTC' in opt.Prediction:
        converter = CTCLabelConverter(opt.character)
    else:
        converter = AttnLabelConverter(opt.character)
    opt.num_class = len(converter.character)

    if opt.rgb:
        opt.input_channel = 3
    model = Model(opt)
    print('model input parameters', opt.imgH, opt.imgW, opt.num_fiducial, opt.input_channel, opt.output_channel,
          opt.hidden_size, opt.num_class, opt.batch_max_length, opt.Transformation, opt.FeatureExtraction,
          opt.SequenceModeling, opt.Prediction)
    model = torch.nn.DataParallel(model).to(device)

    # load model
    print('loading pretrained model from %s' % opt.saved_model)
    model.load_state_dict(torch.load(opt.saved_model, map_location=device))
    opt.exp_name = '_'.join(opt.saved_model.split('/')[1:])
    # print(model)

    """ keep evaluation model and result logs """
    os.makedirs(f'./result/{opt.exp_name}', exist_ok=True)
    os.system(f'cp {opt.saved_model} ./result/{opt.exp_name}/')

    """ setup loss """
    if 'CTC' in opt.Prediction:
        criterion = torch.nn.CTCLoss(zero_infinity=True).to(device)
    else:
        criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)  # ignore [GO] token = ignore index 0

    """ evaluation """
    model.eval()
    with torch.no_grad():
        if opt.benchmark_all_eval:  # evaluation with 10 benchmark evaluation datasets
            benchmark_all_eval(model, criterion, converter, opt)
        else:
            log = open(f'./result/{opt.exp_name}/log_evaluation.txt', 'a')
            AlignCollate_evaluation = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
            eval_data, eval_data_log = hierarchical_dataset(root=opt.eval_data, opt=opt)
            evaluation_loader = torch.utils.data.DataLoader(
                eval_data, batch_size=opt.batch_size,
                shuffle=False,
                num_workers=int(opt.workers),
                collate_fn=AlignCollate_evaluation, pin_memory=True)
            _, accuracy_by_best_model, norm_ED, _, _, _, _, _ = validation(
                model, criterion, evaluation_loader, converter, opt)
            log.write(eval_data_log)
            print(f'Accuracy: {accuracy_by_best_model:0.8f}')
            print(f'Norm ED: {norm_ED:0.8f}')
            
            log.write(f'Accuracy: {accuracy_by_best_model:0.8f}\n')
            log.write(f'Norm ED: {norm_ED:0.8f}\n')
            log.close()


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--eval_data', required=True, help='path to evaluation dataset')
    parser.add_argument('--benchmark_all_eval', action='store_true', help='evaluate 10 benchmark evaluation datasets')
    parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
    parser.add_argument('--batch_size', type=int, default=192, help='input batch size')
    parser.add_argument('--saved_model', required=True, help="path to saved_model to evaluation")
    """ Data processing """
    parser.add_argument('--batch_max_length', type=int, default=25, help='maximum-label-length')
    parser.add_argument('--imgH', type=int, default=32, help='the height of the input image')
    parser.add_argument('--imgW', type=int, default=100, help='the width of the input image')
    parser.add_argument('--rgb', action='store_true', help='use rgb input')
    parser.add_argument('--character', type=str, default='0123456789abcdefghijklmnopqrstuvwxyz', help='character label')
    parser.add_argument('--sensitive', action='store_true', help='for sensitive character mode')
    parser.add_argument('--PAD', action='store_true', help='whether to keep ratio then pad for image resize')
    parser.add_argument('--data_filtering_off', action='store_true', help='for data_filtering_off mode')
    parser.add_argument('--baiduCTC', action='store_true', help='for data_filtering_off mode')
    """ Model Architecture """
    parser.add_argument('--Transformation', type=str, required=True, help='Transformation stage. None|TPS')
    parser.add_argument('--FeatureExtraction', type=str, required=True, help='FeatureExtraction stage. VGG|RCNN|ResNet')
    parser.add_argument('--SequenceModeling', type=str, required=True, help='SequenceModeling stage. None|BiLSTM')
    parser.add_argument('--Prediction', type=str, required=True, help='Prediction stage. CTC|Attn')
    parser.add_argument('--num_fiducial', type=int, default=20, help='number of fiducial points of TPS-STN')
    parser.add_argument('--input_channel', type=int, default=1, help='the number of input channel of Feature extractor')
    parser.add_argument('--output_channel', type=int, default=512,
                        help='the number of output channel of Feature extractor')
    parser.add_argument('--hidden_size', type=int, default=256, help='the size of the LSTM hidden state')

    opt = parser.parse_args()

    """ vocab / character number configuration """
    if opt.sensitive:
        opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

    cudnn.benchmark = True
    cudnn.deterministic = True
    opt.num_gpu = torch.cuda.device_count()

    test(opt)

In [ ]:
!python3 /kaggle/working/AttentionHTR/model/train.py \
  --train_data /kaggle/input/iam-lmdb/IAM_LMDB/trainresult \
  --valid_data /kaggle/input/validation2/validation \
  --select_data "" \
  --batch_ratio "1" \
  --Transformation TPS \
  --FeatureExtraction neural \
  --SequenceModeling BiLSTM \
  --batch_size 16 \
  --Prediction Attn \
  --patience 200 \
  --output_channel 16 \
  --imgW 50 \
  --valInterval 2576
  
